# Quantum Phase Estimation, Iterative
$\newcommand{\bra}[1]{\left\langle{#1}\right|}$
$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$

To economize in qubits, the References below advocate
using the so called iterative Quantum Phase Estimation (iterative qPE).
Whereas the usual qPE uses multiple pointer qubits and gives the
answer in one shot (passage through a single circuit), the iterative 
qPE uses only a single pointer qubit but requires
passage through multiple circuits, with the parameters
of each circuit depending on the final pointer measurement of the previous circuit.
This works because the kickback phases which each power of U
sends to the pointers in the nomal qPE are cummulative: the k'th
pointer gets a
kickback phase which includes the 
kickback phases accrued by all previous pointer qubits.

In this example, we use 

$U = e^{i*rads*\sigma_Z}$

for some Real number $rads$ 
and we use initial state $\ket{0}$, so $e^{i*rads}$ is the 
eigenvalue we seek.

Here are some of the equations used in the code below
        
``` 
for  k in range(num_reps):

    |           H 
    |           exp(i*alpha(k)*sigz)
    U^(2^k)-----@   
    |           H 
    |           measure n(k) here
```

$\alpha(0) = n(0) =0$

$\alpha(k+1) = 2\alpha(k) + \frac{\pi}{2} n(k)$

$\alpha(k) =   \pi 2^{k-2}\sum_{b=0}^{k-1} \frac{n(b)}{2^{b}}$

$rads = \frac{\alpha(num\_reps-1)}{2^{num\_reps-2}}$


References
----------

1. https://arxiv.org/abs/1512.06860 by Google team

2. https://arxiv.org/abs/1605.03590 by Microsoft team



First change your working directory to the qubiter directory in your computer, and add its path to the path environment variable.

In [1]:
import os
import sys
print(os.getcwd())
os.chdir('../../')
print(os.getcwd())
sys.path.insert(0,os.getcwd())

/home/rrtucci/PycharmProjects/qubiter/qubiter/jupyter_notebooks
/home/rrtucci/PycharmProjects/qubiter


In [2]:
from qubiter.SEO_writer import *
from qubiter.SEO_simulator import *
from qubiter.StateVec import *
import numpy as np
import random as ran

loaded OneQubitGate, WITHOUT autograd.numpy


In [3]:
rads = 2*np.pi*(1/16 + 1/8 + 1e-8)
z_axis = 3
num_qbits = 2
num_reps = 15
file_prefix = 'ph_est_iterative'

emb = CktEmbedder(num_qbits, num_qbits)

alpha = 0
ptr_state = 0
ptr_st_list = []
for k in range(num_reps):
    print('--------k=', k)
    # refresh angle alpha to twice its previous value plus
    # \pi/2 times latest measurement of pointer qubit
    alpha = 2*alpha + np.pi*ptr_state/2
    print('rads, alpha/2^(num_reps)=', rads, alpha/(1 << num_reps-2))

    # write circuit
    wr = SEO_writer(file_prefix, emb)
    wr.write_one_qbit_gate(0, OneQubitGate.had2)
    wr.write_one_qbit_gate(0, OneQubitGate.rot_ax, [alpha, z_axis])

    control_pos = 0
    target_pos = 1
    trols = Controls.new_single_trol(num_qbits, control_pos, kind=True)
    wr.write_controlled_one_qbit_gate(
        target_pos, trols, OneQubitGate.rot_ax, [(1 << k)*rads, z_axis])

    wr.write_one_qbit_gate(0, OneQubitGate.had2)
    wr.close_files()

    # simulate circuit
    init_st_vec = StateVec.get_standard_basis_st_vec([0, 0])
    sim = SEO_simulator(file_prefix, num_qbits, init_st_vec)
    StateVec.describe_st_vec_dict(sim.cur_st_vec_dict, 
        print_st_vec=True, do_pp=True, omit_zero_amps=True, show_pp_probs=True)

    # find final state of pointer qubit
    fin_st_vec = sim.cur_st_vec_dict["pure"]
    # dictionary with key=qubit, value=final (P(0), P(1))
    bit_to_probs = StateVec.get_bit_probs(num_qbits, fin_st_vec.get_pd())
    p0, p1 = bit_to_probs[0]
    
    # random float between 0 and 1
    x = ran.random()
    
    if x < p0:
        ptr_state = 0
    else:
        ptr_state = 1
    ptr_st_list.append(ptr_state)
    print('ptr_state=', ptr_state)
print('---------------------')
print('timeline of bit 0 measurements', ptr_st_list)
print("rads, alpha(num_reps-1)/2^(num_reps-2)", rads, alpha/(1 << num_reps-2))


--------k= 0
rads, alpha/2^(num_reps)= 1.1780973079280255 0.0
*********branch= pure
state vector:
ZL convention (Zero bit Last in state tuple)
(00)ZL ( 0.691342 + 0.461940j)	 prob=0.691342
(01)ZL ( 0.308658 - 0.461940j)	 prob=0.308658
total probability of state vector (=one if no measurements)= 1.000000
dictionary with key=qubit, value=(Prob(0), Prob(1))
{0: (0.691342, 0.308658), 1: (1.0, 0.0)}
ptr_state= 0
--------k= 1
rads, alpha/2^(num_reps)= 1.1780973079280255 0.0
*********branch= pure
state vector:
ZL convention (Zero bit Last in state tuple)
(00)ZL ( 0.146447 + 0.353553j)	 prob=0.146447
(01)ZL ( 0.853553 - 0.353553j)	 prob=0.853553
total probability of state vector (=one if no measurements)= 1.000000
dictionary with key=qubit, value=(Prob(0), Prob(1))
{0: (0.146447, 0.853553), 1: (1.0, 0.0)}
ptr_state= 0
--------k= 2
rads, alpha/2^(num_reps)= 1.1780973079280255 0.0
*********branch= pure
state vector:
ZL convention (Zero bit Last in state tuple)
(00)ZL ( 0.500000 - 0.500000j)	 pro